## Второй бот для отправки уведомлений о встречах

In [6]:
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime
from time import sleep


bot = telebot.TeleBot("6637092297:AAE7hUz7mBorW-gP9r_qL_x-3UdEWlPRBpU") # нужен токен вашего бота meetingreminderzi16_notify_bot

# шедулер для отправки сообщений в чат
@bot.message_handler(func = lambda message: 1==1)
def send_remind_message(message):
    # Собираю все встречи на сегодня в ношу их в расписание (думаю с 8:00 до 19:00 нужно делать это каждые 10 минут, в остальное время только в 07:00)
    while True:
        if datetime.now().hour == 7:
            
    
    # 
    
    
    time_meet = datetime.strptime('1900-01-01 14:28', '%Y-%m-%d %H:%M').replace(year=1900, month=1, day=1, microsecond=0)
    print(f'time_meet = {time_meet}')
    while True:
        time_diff = time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
        datenow = datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
        print(time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0))
#         print(f'time_meet = {time_meet}')
        print(f'datenow={datenow}')
        print(f'time_diff={time_diff}')
        if time_diff.days >= 0 and time_diff.seconds >=0 and time_diff.seconds <= 120:
            text = f'Встреча начнется через: {str(time_diff)} секунд'
            break
        elif time_diff.days < 0:
            text = f'Встреча уже началась: {str(-time_diff)} секунд назад'
            break
            
        sleep(10)
#     text = 'ha'    
    bot.send_message(message.chat.id, text)
    print('отправили')

    


In [18]:
datetime.now().hour
datetime.now().day
datetime.now().month
datetime.now().year

1

In [ ]:
import pandas as pd
cnx = sqlite3.connect('file.db')
df = pd.read_sql_query("SELECT * FROM table_name", cnx)


# подключаем базу данных
conn = sqlite3.connect('meeting_reminder.db')

# курсор для работы с таблицами
cursor = conn.cursor()

# try:
#     # sql запрос для создания таблицы
#     query = 'DROP TABLE "meeting_reminder"'
#     # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
#     cursor.execute(query)
# except:
#     pass
#     raise

try:
    # sql запрос для создания таблицы
    query = 'CREATE TABLE "meeting_reminder" ("ID" INTEGER UNIQUE, "chat_id" INTEGER, "meet_time" TEXT, "meet_location" TEXT, "meet_name" TEXT, "meet_text_msg" TEXT, PRIMARY KEY ("ID"))'
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
    
    

In [15]:
type(datetime.now() - datetime.now())

datetime.timedelta

In [7]:
bot.polling(none_stop=True, interval=0)

time_meet = 1900-01-01 14:28:00
0:02:43
datenow=1900-01-01 14:25:17
time_diff=0:02:43
0:02:33
datenow=1900-01-01 14:25:27
time_diff=0:02:33
0:02:23
datenow=1900-01-01 14:25:37
time_diff=0:02:23
0:02:13
datenow=1900-01-01 14:25:47
time_diff=0:02:13
0:02:03
datenow=1900-01-01 14:25:57
time_diff=0:02:03
0:01:53
datenow=1900-01-01 14:26:07
time_diff=0:01:53
отправили


In [3]:
# time_meet = datetime.strptime('1900-01-01 22:25:00', '%Y-%m-%d %H:%M:%S').replace(year=1900, month=1, day=1, microsecond=0)
# time_meet

In [4]:
# datenow = datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
# datenow

In [5]:
# time_meet-datenow